In [4]:
from smart_open import open
import json
import csv
import re
MAX_NUMBER_OF_ROWS = 1000000000
number_of_rows = 0
with open('stream.csv',  mode='w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["subdomain", "domain"])
    for line in open('2019-09-20-1568938006-fdns_cname.json'):
        element = json.loads(line)
        if not re.match("^[a-z]", element['name']):
            continue
            
        arr = element['name'].split('.')
        seperator = "."
        domain = seperator.join(arr[-2:])
        if re.match("^[com\.|co\.|net\.]", domain):
            continue
        subdomain = arr[:-2]
        # if len(subdomain) > 2:
        #     continue
        if len(subdomain) > 1:
            subdomain = seperator.join(subdomain)
        else:
            if len(subdomain) == 0:
             continue
            subdomain = subdomain[0]
        number_of_rows += 1

        csv_writer.writerow([subdomain, domain])
        if number_of_rows == MAX_NUMBER_OF_ROWS:
            pass
            
        if not re.match("^[a-z]", element['value']):
            continue
        arr = element['value'].split('.')
        seperator = "."
        domain = seperator.join(arr[-2:])
        subdomain = arr[:-2]
        if len(subdomain) > 1:
            subdomain = seperator.join(subdomain)
        else:
            if len(subdomain) == 0:
                continue
            subdomain = subdomain[0]
        number_of_rows += 1

        csv_writer.writerow([subdomain, domain])
        if number_of_rows == MAX_NUMBER_OF_ROWS:
            pass

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()
df = spark.read.csv('stream.csv', mode="DROPMALFORMED",inferSchema=True, header = True)

In [7]:
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc

#df.createOrReplaceTempView("dns_requests")
#sqlDF = spark.sql("SELECT count(subdomain) FROM dns_requests ").show()
df.groupBy('domain').agg(countDistinct('subdomain')).orderBy(desc("count(DISTINCT subdomain)")).groupBy("count(DISTINCT subdomain)").count().show(20000)


+-------------------------+--------+
|count(DISTINCT subdomain)|   count|
+-------------------------+--------+
|                  9534421|       1|
|                  5562462|       1|
|                  2526296|       1|
|                  2060336|       1|
|                   946580|       1|
|                   849595|       1|
|                   823218|       1|
|                   680945|       1|
|                   637255|       1|
|                   415571|       1|
|                   376913|       1|
|                   345711|       1|
|                   328810|       1|
|                   325492|       1|
|                   311323|       1|
|                   299266|       1|
|                   283252|       1|
|                   245873|       1|
|                   216419|       1|
|                   185435|       1|
|                   171583|       1|
|                   171202|       1|
|                   170365|       1|
|                   168860|       1|
|